<a href="https://colab.research.google.com/github/Hossam-N/Regex-to-DFA/blob/main/1190097_HossamNabilSaad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uqq ipdb

import ipdb



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.3/793.3 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 KB 46.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 8.11.0 which is incompatible.


In [18]:
from collections import defaultdict
import re
import json
import heapq
import sys
import graphviz
from IPython.display import Image
from base64 import b64encode
from IPython.display import HTML

In [ ]:
expression='ab'

Regular expression to NFA


In [ ]:
string = insert_concat_operator(ab)

NameError: ignored

In [19]:

# steps to convert regex to NFA
# take regex and add concationatipn operator . for convenience through add_concatination func
# the resulted expression after concationation will be an infix exp (operand operator operand)
# convert infix exp to postfix exp through compute_postfix func (postfix: operand operator) (opernad will be arranged according to its precedence)
# convert postfix exp to an NFA 
# using NFA to create json file and graph 


non_symbol=['+','*','.','|','(',')']

epsilon ='ε'

class State:
  name=None
  edge_uno=None
  edge_due=None
  count=None
  
  def __lt__(self,other):
    return self.count < other.count 

class NFA:
  initialState=None
  acceptedState=None

  def __init__(self,initialState,acceptedState):
    self.initialState=initialState
    self.acceptedState=acceptedState


def is_non_symbol(char):
  return char in "+?|*()[]"

def check_operator(char):
  return char in "+.*?|" 


def validate(regex): # check if it is a valid regular expression
  try:
    re.compile(regex)
    return True
  except re.error:
    return False    

def add_concatination(regex):
  if len(regex) == 1:
    return regex + '.'

  re=[]
  prev_char=None

  for char in regex:
    if prev_char is not None:
      curr_op=is_non_symbol(char)
      prev_op=is_non_symbol(prev_char)
      if (not curr_op and not prev_op) or (prev_char in '*+?' and (not curr_op or char == '[' or char == '(')) or (prev_char == ')' and (not curr_op or char == '(')) or (prev_char == ']' and (not curr_op or char == '[' )) or (not prev_op and char in '(['):
        re.append('.')
    re.append(char)
    prev_char=char  

  return ''.join(re) 

def comp_precedence(op):
  pr= {
      '*':2,
      '+':2,
      '?':2,
      '|':1,
      '.':0
  }  
  return pr[op]  

def compute_postfix(exp):
  operators=[]
  postfix=[]

  for i,char in enumerate(exp):
        flag=True
        for j in range(i,len(exp)):
            if exp[j]=='[':
                break
            elif exp[j]==']':
                flag=False
                break
        if flag:
            if char==']':
                continue 
            elif char.isalnum():
                postfix.append(char)
            elif char == '[':  
                for j in range(i,len(exp)):
                    postfix.append(exp[j])
                    if exp[j]==']':
                        break
                continue
            elif check_operator(char):
                if operators and check_operator(operators[-1]) and comp_precedence(operators[-1])>= comp_precedence(char):
                    while operators and check_operator(operators[-1]) and comp_precedence(operators[-1])>= comp_precedence(char):
                        postfix.append(operators.pop())
                    operators.append(char)
                elif not operators or operators[-1] == '(':
                    operators.append(char)    
                else:
                    operators.append(char)
            elif char == '(':
                operators.append(char)
            elif char == ')':
                while operators and operators[-1] not in '(': 
                   postfix.append(operators.pop())
                operators.pop()
   
  while operators:
      postfix.append(operators.pop())
  

  return ''.join(postfix)



# def format_regex(regex):
#     re = add_concat(regex)
#     ree = compute_postfix(re)
#     return ree

def convert_postfix_to_NFA(exp):
  stack=[]
  count=0
  inp=''
  flag=False
  for char in exp:
    if char ==  ']':
      flag=False
      inp+=char
      startState=State()
      acceptedState=State()
      startState.name=inp
      startState.edge_uno=acceptedState
      startState.count=count
      acceptedState.count=count+1
      count+=2
      newNFA=NFA(startState,acceptedState)
      stack.append(newNFA)
      inp=''
    elif char ==  '[' or flag:
      flag=True
      if(char != '.'):
        inp+=char
      continue  
    elif char == '.':
      ovalN=stack.pop()
      ovalM=stack.pop()
      ovalM.acceptedState.edge_uno=ovalN.initialState
      newNFA=NFA(ovalM.initialState,ovalN.acceptedState)
      stack.append(newNFA)
    elif char =='|':
      ovalN=stack.pop()
      ovalM=stack.pop()
      startState=State()
      acceptedState=State()
      startState.edge_uno=ovalM.initialState
      startState.edge_due=ovalN.initialState
      ovalM.acceptedState.edge_uno=acceptedState
      ovalN.acceptedState.edge_uno=acceptedState
      startState.count=count
      acceptedState.count=count+1
      count+=2
      newNFA=NFA(startState,acceptedState)
      stack.append(newNFA)
    elif char ==  '*':
      ovalN=stack.pop()
      startState=State()
      acceptedState=State()
      ovalN.acceptedState.edge_uno=startState
      ovalN.acceptedState.edge_due=acceptedState
      startState.edge_uno=acceptedState
      startState.edge_due=ovalN.initialState
      startState.count=count
      acceptedState.count=count+1
      count+=2
      newNFA=NFA(startState,acceptedState)
      stack.append(newNFA)
    elif char ==  '+':
      ovalN=stack.pop()
      startState=State()
      acceptedState=State()
      ovalN.acceptedState.edge_uno=startState
      ovalN.acceptedState.edge_due=acceptedState
      startState.edge_uno=ovalN.initialState
      startState.count=count
      acceptedState.count=count+1
      count+=2
      newNFA=NFA(startState,acceptedState)
      stack.append(newNFA)
    elif char ==  '?':
      ovalN=stack.pop()
      startState=State()
      acceptedState=State()
      startState.edge_uno=ovalN.initialState
      startState.edge_due=acceptedState
      ovalN.acceptedState.edge_uno=acceptedState
      startState.count=count
      acceptedState.count=count+1
      count+=2
      newNFA=NFA(startState,acceptedState)
      stack.append(newNFA)
    else: 
      startState=State()
      acceptedState=State()
      startState.name=char
      startState.edge_uno=acceptedState
      startState.count=count
      acceptedState.count=count+1
      count+=2
      stack.append(NFA(startState,acceptedState))
  return stack.pop()


def createJSON(NFA):
    states  = defaultdict()
    visited = defaultdict(bool)
    stack = []
    stack.append(NFA.initialState)
    states['startingState'] = 'S' + str(NFA.initialState.count)
    states['terminatingState'] = 'S' + str(NFA.acceptedState.count)
    while stack:
        heapq.heapify(stack)
        state_edge = heapq.heappop(stack)
        name = state_edge.name
        if name is None:
            name = epsilon
        checkFirst = state_edge.edge_uno != None
        checkSecond = state_edge.edge_due != None
        if checkFirst and checkSecond:
            states['S' + str(state_edge.count)] = {
                'isTerminatingState': state_edge.count == NFA.acceptedState.count,
                 'S' + str(state_edge.edge_uno.count): name,
                 'S' + str(state_edge.edge_due.count): name
            }
        elif checkFirst:
            states['S' + str(state_edge.count)] = {
                'isTerminatingState': state_edge.count == NFA.acceptedState.count,
                'S' + str(state_edge.edge_uno.count) : name,
            }
        elif checkSecond:
            states['S' + str(state_edge.count)] = {
                'isTerminatingState': state_edge.count == NFA.acceptedState.count,
                 'S' + str(state_edge.edge_due.count): name 
            }
        else:
            states['S' + str(state_edge.count)] = {
                'isTerminatingState': state_edge.count == NFA.acceptedState.count,
            }

        if checkFirst and not visited [state_edge.edge_uno]:
            stack.append(state_edge.edge_uno)
            visited[state_edge.edge_uno] = True
        if checkSecond and not visited[state_edge.edge_due]:
            stack.append(state_edge.edge_due)
            visited[state_edge.edge_due] = True
        
    json_object = json.dumps(states, indent = 4)
  
    with open("NFA.json", "w") as outfile:
        outfile.write(json_object)
        print('output written in NFA.json')
        return json_object, states

def createNFAGraph(NFA):
    dot = graphviz.Digraph("NFA", format="png")
    dot.attr(rankdir="LR")
    visited = defaultdict(bool)
    visited_edge = defaultdict(bool)
    stack = []
    stack.append(NFA.initialState)
    dot.node('S' + str(NFA.initialState.count), shape = 'circle', style = 'filled', fillcolor = 'green')
    while stack:
        heapq.heapify(stack)
        edge = heapq.heappop(stack)

        label = 'S' + str(edge.count)
        name = edge.name
        if name is None:
            name = epsilon

        checkFirst = edge.edge_uno != None
        checkSecond = edge.edge_due != None

        if checkFirst:
            state_name = 'S' + str(edge.edge_uno.count)
            if edge.edge_uno.count == NFA.acceptedState.count:
                dot.node(state_name, shape = 'doublecircle', style = 'filled', fillcolor = 'orange')
            else:
                dot.node(state_name, shape = 'circle')

            if not visited_edge[label + state_name]:
                dot.edge(label, state_name, label = name)
                visited_edge[label + state_name] = True

            if not visited[edge.edge_uno]:
                visited[edge.edge_uno] = True
                stack.append(edge.edge_uno)

        if checkSecond:
            state_name = 'S' + str(edge.edge_due.count)
            if edge.edge_due.count == NFA.acceptedState.count:
                dot.node(state_name, shape = 'doublecircle', style = 'filled', fillcolor = 'orange')
            else:
                dot.node(state_name, shape = 'circle')

            if not visited_edge[label + state_name]:
                dot.edge(label, state_name, label = name)
                visited_edge[label + state_name] = True

            if not visited[edge.edge_due]:
                visited[edge.edge_due] = True
                stack.append(edge.edge_due)

    dot.view()






if __name__ == "__main__":
  # ipdb.set_trace()
  reg='(ab)+'
  jsonDict=''
  if validate(reg):
    exp=add_concatination(reg)
    postfix=compute_postfix(exp)
    NFA=convert_postfix_to_NFA(postfix)
    json,jsonDict=createJSON(NFA)
    createNFAGraph(NFA)
  else:  
    print('Invalid Expression')  


output written in NFA.json


NFA to minimized DFA


In [21]:


def extract_states_from_json(nfaData):
  ipdb.set_trace()
  startState=nfaData['startingState']
  acceptingState=nfaData['terminatingState']
  count=int(acceptingState[1:])+1

  stateList=[None]* count

  for i in range(count):
    tempData=nfaData['S'+str(i)]
    tempState=State()

    tempState.count=i
    for k in tempData:
      if k!= 'isTerminatingState':
        name=tempData[k]
        if name != epsilon:
          tempState.name=name
        break
    stateList[i]=tempState      
   
  for i in range(count):
    tempdata=nfaData['S'+ str(i)]
    for j in tempData:
      if j != 'isTerminatingState':
        if stateList[i].edge_uno is None:
          stateList[i].edge_uno=stateList[int(j[1:])]
        elif stateList[i].edge_uno is None:
          stateList[i].edge_due=stateList[int(j[1:])]      

  return stateList,int(startState[1:]),int(acceptingState[1:])  


extract_states_from_json(jsonDict)

> <ipython-input-21-9fce74b93223>(3)extract_states_from_json()
      2   ipdb.set_trace()
----> 3   startState=nfaData['startingState']
      4   acceptingState=nfaData['terminatingState']

ipdb> n
> <ipython-input-21-9fce74b93223>(4)extract_states_from_json()
      3   startState=nfaData['startingState']
----> 4   acceptingState=nfaData['terminatingState']
      5   count=int(acceptingState[1:])+1

ipdb> n
> <ipython-input-21-9fce74b93223>(5)extract_states_from_json()
      4   acceptingState=nfaData['terminatingState']
----> 5   count=int(acceptingState[1:])+1
      6 

ipdb> n
> <ipython-input-21-9fce74b93223>(7)extract_states_from_json()
      6 
----> 7   stateList=[None]* count
      8 

ipdb> n
> <ipython-input-21-9fce74b93223>(9)extract_states_from_json()
      8 
----> 9   for i in range(count):
     10     tempData=nfaData['S'+str(i)]

ipdb> n
> <ipython-input-21-9fce74b93223>(10)extract_states_from_json()
      9   for i in range(count):
---> 10     tempData=nfaData['S'+str(

BdbQuit: ignored